In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('IWI20-21data.txt')
winter2 = pd.read_csv ('IWI21-22data.txt')
winter3 = pd.read_csv ('IWI22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
# eliminate special weather
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
iwi = LocationInfo(
    'Wiscasset', 'Maine', 
    latitude= 44.0029,
    longitude= -69.6656,
    timezone='America/Connecticut'
)

iwi

LocationInfo(name='Wiscasset', region='Maine', timezone='America/Connecticut', latitude=44.0029, longitude=-69.6656)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(iwi.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(iwi.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(iwi.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:54:11  2020-12-01 16:01:25  2020-12-01
1   2020-12-02 06:55:17  2020-12-02 16:01:07  2020-12-02
2   2020-12-03 06:56:21  2020-12-03 16:00:51  2020-12-03
3   2020-12-04 06:57:24  2020-12-04 16:00:38  2020-12-04
4   2020-12-05 06:58:25  2020-12-05 16:00:27  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:23:40  2021-02-24 17:20:35  2021-02-24
86  2021-02-25 06:22:01  2021-02-25 17:21:54  2021-02-25
87  2021-02-26 06:20:22  2021-02-26 17:23:13  2021-02-26
88  2021-02-27 06:18:42  2021-02-27 17:24:32  2021-02-27
89  2021-02-28 06:17:01  2021-02-28 17:25:51  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:53:55  2021-12-01 16:01:29  2021-12-01
1   2021-12-02 06:55:01  2021-12-02 16:01:11  2021-12-02
2   2021-12-03 06:56:06  2021-12-03 16:00:54  2021-12-03
3   2021-12-04 06:57:09  2021-12-04 16:00:41  2021-12-04
4   2021-12-05 06:58:11  2021-12-05 16:00:29  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:24:03  2022-02-24 17:20:16  2022-02-24
86  2022-02-25 06:22:25  2022-02-25 17:21:35  2022-02-25
87  2022-02-26 06:20:46  2022-02-26 17:22:54  2022-02-26
88  2022-02-27 06:19:06  2022-02-27 17:24:13  2022-02-27
89  2022-02-28 06:17:26  2022-02-28 17:25:32  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:53:39  2022-12-01 16:01:34  2022-12-01
1   2022-12-02 06:54:45  2022-12-02 16:01:15  2022-12-02
2   2022-12-03 06:55:50  2022-12-03 16:00:58  2022-12-03
3   2022-12-04 06:56:54  2022-12-04 16:00:43  2022-12-04
4   2022-12-05 06:57:56  2022-12-05 16:00:32  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:24:27  2023-02-24 17:19:56  2023-02-24
86  2023-02-25 06:22:49  2023-02-25 17:21:16  2023-02-25
87  2023-02-26 06:21:10  2023-02-26 17:22:35  2023-02-26
88  2023-02-27 06:19:31  2023-02-27 17:23:54  2023-02-27
89  2023-02-28 06:17:50  2023-02-28 17:25:13  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IWI,2020-12-09 23:53:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:02:15,2020-12-09 16:00:11
1,IWI,2020-12-10 01:53:00,0.0001,-SN BR,2020-12-10,2020-12-10 07:03:08,2020-12-10 16:00:13
2,IWI,2020-12-10 02:53:00,0.0001,-SN BR,2020-12-10,2020-12-10 07:03:08,2020-12-10 16:00:13
3,IWI,2020-12-14 22:53:00,0.0001,-SN,2020-12-14,2020-12-14 07:06:23,2020-12-14 16:00:47
4,IWI,2020-12-17 11:53:00,0.0001,-SN,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40
...,...,...,...,...,...,...,...
137,IWI,2021-02-20 04:53:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:30:04,2021-02-20 17:15:15
138,IWI,2021-02-20 05:53:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:30:04,2021-02-20 17:15:15
139,IWI,2021-02-22 16:53:00,0.0001,-SN,2021-02-22,2021-02-22 06:26:53,2021-02-22 17:17:55
140,IWI,2021-02-22 19:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 06:26:53,2021-02-22 17:17:55


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IWI,2021-12-09 06:53:00,0.0001,-SN,2021-12-09,2021-12-09 07:02:02,2021-12-09 16:00:10
1,IWI,2021-12-09 07:53:00,0.0001,-SN,2021-12-09,2021-12-09 07:02:02,2021-12-09 16:00:10
2,IWI,2021-12-10 12:53:00,0.0001,-SN,2021-12-10,2021-12-10 07:02:56,2021-12-10 16:00:12
3,IWI,2021-12-10 13:53:00,0.0001,-SN,2021-12-10,2021-12-10 07:02:56,2021-12-10 16:00:12
4,IWI,2021-12-10 14:53:00,0.0001,-SN BR,2021-12-10,2021-12-10 07:02:56,2021-12-10 16:00:12
...,...,...,...,...,...,...,...
116,IWI,2022-02-25 20:53:00,0.2500,-SN,2022-02-25,2022-02-25 06:22:25,2022-02-25 17:21:35
117,IWI,2022-02-25 21:53:00,0.0001,-SN,2022-02-25,2022-02-25 06:22:25,2022-02-25 17:21:35
118,IWI,2022-02-25 22:53:00,0.0001,-SN,2022-02-25,2022-02-25 06:22:25,2022-02-25 17:21:35
119,IWI,2022-02-25 23:53:00,0.0001,-SN,2022-02-25,2022-02-25 06:22:25,2022-02-25 17:21:35


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IWI,2022-12-12 00:53:00,0.0001,-SN,2022-12-12,2022-12-12 07:04:26,2022-12-12 16:00:20
1,IWI,2022-12-12 01:53:00,0.0001,-SN,2022-12-12,2022-12-12 07:04:26,2022-12-12 16:00:20
2,IWI,2022-12-12 02:53:00,0.0001,-SN,2022-12-12,2022-12-12 07:04:26,2022-12-12 16:00:20
3,IWI,2022-12-12 03:53:00,0.5100,-SN,2022-12-12,2022-12-12 07:04:26,2022-12-12 16:00:20
4,IWI,2022-12-12 04:53:00,0.0001,-SN,2022-12-12,2022-12-12 07:04:26,2022-12-12 16:00:20
...,...,...,...,...,...,...,...
148,IWI,2023-02-26 18:53:00,0.7600,-SN BR,2023-02-26,2023-02-26 06:21:10,2023-02-26 17:22:35
149,IWI,2023-02-26 19:53:00,0.5100,-SN BR,2023-02-26,2023-02-26 06:21:10,2023-02-26 17:22:35
150,IWI,2023-02-26 20:53:00,0.5100,-SN BR,2023-02-26,2023-02-26 06:21:10,2023-02-26 17:22:35
151,IWI,2023-02-26 21:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:21:10,2023-02-26 17:22:35


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
4,IWI,2020-12-17 11:53:00,0.0001,-SN,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40
5,IWI,2020-12-17 12:53:00,0.7600,SN FZFG,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40
6,IWI,2020-12-17 13:53:00,1.2700,SN FZFG,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40
7,IWI,2020-12-17 14:53:00,1.5200,SN FZFG,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40
8,IWI,2020-12-17 15:53:00,2.2900,SN FZFG,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40
18,IWI,2020-12-22 07:53:00,0.2500,-SN BR,2020-12-22,2020-12-22 07:11:13,2020-12-22 16:03:56
21,IWI,2021-01-02 07:53:00,1.7800,-SN BR,2021-01-02,2021-01-02 07:13:45,2021-01-02 16:12:12
22,IWI,2021-01-02 08:53:00,0.7600,-SN BR,2021-01-02,2021-01-02 07:13:45,2021-01-02 16:12:12
23,IWI,2021-01-02 09:53:00,0.7600,-SN,2021-01-02,2021-01-02 07:13:45,2021-01-02 16:12:12
24,IWI,2021-01-02 15:53:00,2.0300,-SN BR,2021-01-02,2021-01-02 07:13:45,2021-01-02 16:12:12


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
4,IWI,2020-12-17 11:53:00,0.0001,-SN,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40,2020-12-17 08:08:28,2020-12-17 15:01:40
5,IWI,2020-12-17 12:53:00,0.7600,SN FZFG,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40,2020-12-17 08:08:28,2020-12-17 15:01:40
6,IWI,2020-12-17 13:53:00,1.2700,SN FZFG,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40,2020-12-17 08:08:28,2020-12-17 15:01:40
7,IWI,2020-12-17 14:53:00,1.5200,SN FZFG,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40,2020-12-17 08:08:28,2020-12-17 15:01:40
22,IWI,2021-01-02 08:53:00,0.7600,-SN BR,2021-01-02,2021-01-02 07:13:45,2021-01-02 16:12:12,2021-01-02 08:13:45,2021-01-02 15:12:12
23,IWI,2021-01-02 09:53:00,0.7600,-SN,2021-01-02,2021-01-02 07:13:45,2021-01-02 16:12:12,2021-01-02 08:13:45,2021-01-02 15:12:12
26,IWI,2021-01-05 12:53:00,0.0001,-SN BR,2021-01-05,2021-01-05 07:13:34,2021-01-05 16:15:08,2021-01-05 08:13:34,2021-01-05 15:15:08
27,IWI,2021-01-05 13:53:00,0.2500,-SN BR,2021-01-05,2021-01-05 07:13:34,2021-01-05 16:15:08,2021-01-05 08:13:34,2021-01-05 15:15:08
28,IWI,2021-01-05 14:53:00,0.5100,-SN BR,2021-01-05,2021-01-05 07:13:34,2021-01-05 16:15:08,2021-01-05 08:13:34,2021-01-05 15:15:08
41,IWI,2021-01-22 08:53:00,0.0001,-SN,2021-01-22,2021-01-22 07:05:37,2021-01-22 16:35:30,2021-01-22 08:05:37,2021-01-22 15:35:30


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.27565
2021-2022: 0.46375217391304346
2022-2023: 0.5006323529411765


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IWI,2020-12-09 23:53:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:02:15,2020-12-09 16:00:11
1,IWI,2020-12-10 01:53:00,0.0001,-SN BR,2020-12-10,2020-12-10 07:03:08,2020-12-10 16:00:13
2,IWI,2020-12-10 02:53:00,0.0001,-SN BR,2020-12-10,2020-12-10 07:03:08,2020-12-10 16:00:13
3,IWI,2020-12-14 22:53:00,0.0001,-SN,2020-12-14,2020-12-14 07:06:23,2020-12-14 16:00:47
9,IWI,2020-12-17 16:53:00,2.0300,+SN FZFG,2020-12-17,2020-12-17 07:08:28,2020-12-17 16:01:40
...,...,...,...,...,...,...,...
136,IWI,2021-02-20 03:53:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:30:04,2021-02-20 17:15:15
137,IWI,2021-02-20 04:53:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:30:04,2021-02-20 17:15:15
138,IWI,2021-02-20 05:53:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:30:04,2021-02-20 17:15:15
140,IWI,2021-02-22 19:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 06:26:53,2021-02-22 17:17:55


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_40140\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.3915104166666667
2021-2022: 0.7440333333333334
2022-2023: 0.3406914893617021


In [52]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'ME'

data.to_csv('SO.iwi.csv', index=False)